In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [3]:
import parkit as pk

pk.import_site('C:\\Users\\rdpuser\\Desktop\\test')

In [4]:
# Smoke Tests

In [5]:
smoke_tests = pk.bind_task('smoke_tests')

In [6]:
pk.pool().start()

In [ ]:
trace = smoke_tests(1000)
pk.wait(trace, lambda: trace.done)
print(trace)

In [ ]:
pk.pool().stop()

In [ ]:
# File Tests

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
rows = []
for i in range(10000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [ ]:
f = pk.File(mode = 'wb')

In [ ]:
with f:
    df.to_feather(f)

In [ ]:
f.mode = 'rb'
with f:
    assert df.equals(pd.read_feather(f))

In [ ]:
a = np.zeros((100,100))

In [ ]:
f = pk.File(mode = 'wb+')
with f:
    np.save(f, a)

In [ ]:
f.mode = 'rb+'
with f:
    assert np.array_equal(a, np.load(f))